In [ ]:
package_paths = [
    '../input/pytorch-image-models/pytorch-image-models-master', #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
    '../input/image-fmix/FMix-master'
]
import sys; 

for pth in package_paths:
    sys.path.append(pth)
    
from fmix import sample_mask, make_low_freq_image, binarise_mask

In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import timm

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
#from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from torchvision.datasets import ImageFolder

In [ ]:
CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'epochs': 10,
    'train_bs': 16,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0'
}

> We could do stratified validation split in each fold to make each fold's train and validation set looks like the whole train set in target distributions.

# Helper Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

img = get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
plt.imshow(img)
plt.show()

# # Define Train\Validation Image Augmentations

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_valid_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.430, 0.497, 0.314], std=[0.211, 0.217, 0.193], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# test dataset


In [ ]:
class test_dataset(Dataset):
    def __init__(self,transforms=None):
        self.transforms = transforms
        self.image_name = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')['image_id']
    
    
    def __len__(self):
        return len(self.image_name)
    
    
    def __getitem__(self,index):
        img_name = self.image_name[index]
        img = get_img('../input/cassava-leaf-disease-classification/test_images/{}'.format(img_name))
        if self.transforms:
            img = self.transforms(image=img)['image']
        return index,img, img_name


# Model

In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)

        '''self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
            )'''
        

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
if __name__ == '__main__':
    # bulid test dataset and dataloader
    test_transforms = get_valid_transforms()
    test_ds = test_dataset(test_transforms)
    test_dataloader = torch.utils.data.DataLoader(
        test_ds, 
        batch_size=1,
        num_workers=0,
        shuffle=False,
        pin_memory=False,
    )
    # bulid model and load model state
    device = torch.device('cuda:0')
    model = CassvaImgClassifier(model_arch=CFG['model_arch'],n_class=5).to(device)
    model.load_state_dict(torch.load('../input/pytorch-efficientnet-baseline-train-amp-aug/tf_efficientnet_b4_ns_fold_4_4'))
    submission = pd.DataFrame(columns=('image_id','label'))
    for i, img ,img_name in test_dataloader:
        model.eval()
        img = img.to(device)
        index = i.item()
        label = torch.argmax(model(img)).item()
        submission = submission.append([{'image_id':img_name[0],'label':label}],ignore_index=True)
    submission.to_csv('submission.csv',index=False)